In [9]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
# Install necessary libraries for transformers and NLP
!pip install -q transformers datasets torch accelerate
!pip install transformers datasets torch accelerate

In [10]:
import pandas as pd

# Load the provided datasets
train_path = "/kaggle/input/datascienee/train (15).csv"
test_path = "/kaggle/input/datascienee/test (15).csv"
#submission_path = "/mnt/data/SampleSubmission (13).csv"

# Read the datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
#submission_df = pd.read_csv(submission_path)

# Display the first few rows of each dataset to understand the structure
train_df.head(), test_df.head()


(      id                                          title  \
 0  11098                            Getting of the weed   
 1    116  Seeking advice on how to face a hospital stay   
 2   7189              Trying to re-enter the work place   
 3   4350                       Family history of cancer   
 4   9749     Inappropriate relationship with therapist.   
 
                                              content  \
 0  [Post removed at request of member]\nHi, welco...   
 1  Hi NMTB,\nThanks for your post. \nI think a lo...   
 2  Hello, \nI’m Cas and for a fair while now I ha...   
 3  Hey everyone.\nI've just discovered that anoth...   
 4  Hi everyone\nI guess the title says it all rea...   
 
                             target  
 0  suicidal-thoughts-and-self-harm  
 1                          anxiety  
 2                          anxiety  
 3                          anxiety  
 4                       depression  ,
       id                                              title  \
 

In [12]:
# Fill missing values
train_df["content"].fillna(train_df["title"], inplace=True)
test_df["content"].fillna(test_df["title"], inplace=True)

# Encode target labels
label_encoder = LabelEncoder()
train_df["label"] = label_encoder.fit_transform(train_df["target"])

<ipython-input-12-99b5ab0f5ffc>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df["content"].fillna(train_df["title"], inplace=True)
<ipython-input-12-99b5ab0f5ffc>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

In [13]:
# Train-Validation Split (80% train, 20% validation)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df["content"], train_df["label"], test_size=0.2, random_state=42
)

In [14]:
# Load tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenization function
def tokenize_data(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True, max_length=256)

# Convert to Hugging Face dataset format
train_dataset = Dataset.from_dict({"content": train_texts.tolist(), "label": train_labels.tolist()})
val_dataset = Dataset.from_dict({"content": val_texts.tolist(), "label": val_labels.tolist()})

# Apply tokenization
train_dataset = train_dataset.map(tokenize_data, batched=True)
val_dataset = val_dataset.map(tokenize_data, batched=True)

# Load pre-trained RoBERTa model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_encoder.classes_))

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,  # Include validation dataset
)


Map:   0%|          | 0/17720 [00:00<?, ? examples/s]

Map:   0%|          | 0/4431 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

In [ ]:
# Save model
model.save_pretrained("roberta_model")
tokenizer.save_pretrained("roberta_model")

In [ ]:
# Predict on test set
test_encodings = tokenizer(list(test_df["content"]), truncation=True, padding=True, max_length=256, return_tensors="pt")
test_inputs = {key: value for key, value in test_encodings.items()}

In [ ]:
# Get predictions
model.eval()
with torch.no_grad():
    outputs = model(**test_inputs)
predictions = torch.argmax(outputs.logits, dim=-1).numpy()

In [ ]:
# Convert predictions back to labels
test_df["target"] = label_encoder.inverse_transform(predictions)

# Save submission
test_df[["id", "target"]].to_csv("enova_1st.csv", index=False)
print("Submission saved!")